In [1]:
import bayesfast as bf
import numpy as np

In [2]:
D = 16 # number of dims
a = 1.
b = 0.5
lower = np.full(D, -30.) # lower bound of the prior for x_1, ...
upper = np.full(D, 30.) # upper bound of the prior for x_1, ...
lower[0] = -4 # lower bound of the prior for x_0
upper[0] = 4 # upper bound of the prior for x_0
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

def logp(x):
    n = x.shape[-1]
    _a = -0.5 * x[..., 0]**2 / a**2
    _b = -0.5 * np.sum(x[..., 1:]**2, axis=-1) * np.exp(-2 * b * x[..., 0])
    _c = (-0.5 * np.log(2 * np.pi * a**2) - 
          0.5 * (n - 1) * np.log(2 * np.pi) - (n - 1) * b * x[..., 0])
    return _a + _b + _c - const

def grad(x):
    n = x.shape[-1]
    foo = -x / np.insert(np.full((*x.shape[:-1], n - 1), 
                                 np.exp(2 * b * x[..., 0])), 0, a**2, axis=-1)
    foo[0] += b * np.sum(x[..., 1:]**2, axis=-1) * np.exp(-2 * b * x[..., 0])
    foo[0] -= (n - 1) * b
    return foo

In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33213 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 126.70 GB


In [4]:
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace,
                'random_state': 0}, post={}, client=client)

In [5]:
rec.run()

/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #7 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #7 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #7 : overflow encountered in exp
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #2 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarnin

 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 1.78 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 1.77 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.29 seconds. (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.05 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.01 seconds. (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.51 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.18 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 2.46 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 1.70 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 1.95 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 1000 / 2500 ], last 500 samples us

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

 ***** PostStep finished. ***** 



In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -63.4988

(-63.49564469101636, 0.01605608763046655)